In [1]:
from requests import get
from bs4 import BeautifulSoup
from dataclasses import dataclass
import json

In [2]:
ADRESSE = "https://www.tennisendirect.net/atp/classement/"

reponse = get(ADRESSE)
print(reponse.status_code)
assert reponse.status_code == 200


200


In [16]:
classement_soupe = BeautifulSoup(reponse.text, features="lxml")
classement_soupe

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html lang="fr" xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<title>Classement ATP 04.11.2024 | TennisEnDirect.fr</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Classement ATP 04.11.2024" name="description"/>
<meta content="eHM, s.r.o." name="author"/>
<meta content="index,follow" name="robots"/>
<link href="https://www.tennisendirect.net/styles/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://www.tennisendirect.net/styles/main2.css?v=2.118" rel="stylesheet" type="text/css"/>
<!--[if lt IE 7]>
        <script src="./scripts/png-fix.js" type="text/javascript" defer="defer"></script>
    <![endif]-->
<script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.2/jquery.min.js" type="t

In [4]:
tables = classement_soupe.find_all("table", attrs={"class": "table_pranks"})
assert len(tables) == 2

In [5]:
tables

[<table class="table_pranks">
 <tr class="header"><td class="w20">#</td><td></td><td class="w50">Points</td></tr><tr class="pair"><td class="w20">1.</td><td><img alt="Italy" height="16" src="https://www.tennisendirect.net/flags/flag_italy.png" width="16"/> <a href="https://www.tennisendirect.net/atp/jannik-sinner/" title="Jannik Sinner">Jannik Sinner</a> (ITA) (23 ans)</td><td class="w50">11330</td></tr><tr class="unpair"><td class="w20">2.</td><td><img alt="Germany" height="16" src="https://www.tennisendirect.net/flags/flag_germany.png" width="16"/> <a href="https://www.tennisendirect.net/atp/alexander-zverev/" title="Alexander Zverev">Alexander Zverev</a> (DEU) (27 ans)</td><td class="w50">7715</td></tr><tr class="pair"><td class="w20">3.</td><td><img alt="Spain" height="16" src="https://www.tennisendirect.net/flags/flag_spain.png" width="16"/> <a href="https://www.tennisendirect.net/atp/carlos-alcaraz-garfia/" title="Carlos Alcaraz">Carlos Alcaraz</a> (ESP) (21 ans)</td><td class="w

In [6]:
table_inter, _ = tables
table_inter

<table class="table_pranks">
<tr class="header"><td class="w20">#</td><td></td><td class="w50">Points</td></tr><tr class="pair"><td class="w20">1.</td><td><img alt="Italy" height="16" src="https://www.tennisendirect.net/flags/flag_italy.png" width="16"/> <a href="https://www.tennisendirect.net/atp/jannik-sinner/" title="Jannik Sinner">Jannik Sinner</a> (ITA) (23 ans)</td><td class="w50">11330</td></tr><tr class="unpair"><td class="w20">2.</td><td><img alt="Germany" height="16" src="https://www.tennisendirect.net/flags/flag_germany.png" width="16"/> <a href="https://www.tennisendirect.net/atp/alexander-zverev/" title="Alexander Zverev">Alexander Zverev</a> (DEU) (27 ans)</td><td class="w50">7715</td></tr><tr class="pair"><td class="w20">3.</td><td><img alt="Spain" height="16" src="https://www.tennisendirect.net/flags/flag_spain.png" width="16"/> <a href="https://www.tennisendirect.net/atp/carlos-alcaraz-garfia/" title="Carlos Alcaraz">Carlos Alcaraz</a> (ESP) (21 ans)</td><td class="w50

In [7]:
def extraire_lignes(table):
    return table.find_all(
        "tr", class_=lambda class_name: class_name in ["pair", "unpair"]
    )

In [8]:
@dataclass
class Ligne:
    rank: str
    pays: str
    lien_joueur: str
    nom_joueur: str
    pays_abreviation: str
    age: str
    points: str

In [19]:
def genere_ligne(ligne):
    colonnes = [td.text.strip() for td in ligne.find_all("td")]
    if len(colonnes) == 3:
        rank, joueur_info, points = colonnes
        print(rank, joueur_info, points)
    else:
        print("Format inattendu dans la ligne:", colonnes)
        return None

    try:
        lien_joueur = ligne.find("a")["href"]
        nom_joueur = ligne.find("a")["title"]
        pays_abreviation = joueur_info.split("(")[1].split(")")[0]
        pays = ligne.find("img")["alt"]
        age = joueur_info.split("(")[2].split(")")[0] if "(" in joueur_info else ""
    except (IndexError, AttributeError):
        lien_joueur, pays, pays_abreviation, nom_joueur, age = (
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
        print("Problème d'extraction d'une ligne:", ligne)

    return Ligne(
        rank=rank,
        pays=pays,
        lien_joueur=lien_joueur,
        nom_joueur=nom_joueur,
        pays_abreviation=pays_abreviation,
        age=age,
        points=points,
    )

In [11]:
lignes_inter = extraire_lignes(table_inter)

In [20]:
joueurs = [genere_ligne(ligne) for ligne in lignes_inter if genere_ligne(ligne)]

1. Jannik Sinner (ITA) (23 ans) 11330
1. Jannik Sinner (ITA) (23 ans) 11330
2. Alexander Zverev (DEU) (27 ans) 7715
2. Alexander Zverev (DEU) (27 ans) 7715
3. Carlos Alcaraz (ESP) (21 ans) 7210
3. Carlos Alcaraz (ESP) (21 ans) 7210
4. Daniil Medvedev (RUS) (28 ans) 5230
4. Daniil Medvedev (RUS) (28 ans) 5230
5. Novak Djokovic (SRB) (37 ans) 5210
5. Novak Djokovic (SRB) (37 ans) 5210
6. Taylor Harry Fritz (USA) (27 ans) 4300
6. Taylor Harry Fritz (USA) (27 ans) 4300
7. Casper Ruud (NOR) (25 ans) 3855
7. Casper Ruud (NOR) (25 ans) 3855
8. Alex De Minaur (AUS) (25 ans) 3745
8. Alex De Minaur (AUS) (25 ans) 3745
9. Andrey Rublev (RUS) (27 ans) 3720
9. Andrey Rublev (RUS) (27 ans) 3720
10. Grigor Dimitrov (BGR) (33 ans) 3340
10. Grigor Dimitrov (BGR) (33 ans) 3340
11. Holger Rune (DEN) (21 ans) 3225
11. Holger Rune (DEN) (21 ans) 3225
12. Stefanos Tsitsipas (GRC) (26 ans) 3165
12. Stefanos Tsitsipas (GRC) (26 ans) 3165
13. Tommy Paul (USA) (27 ans) 3145
13. Tommy Paul (USA) (27 ans) 3145
14